# ItaData2024

## COPD

propositional analysis

In [1]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots

  Activating project at `~/Documents/Aclai/audio-rules2024`


### Settings

In [2]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

features = :catch9
# features = :minmax
# features = :custom

loadset = false
# loadset = true

scale = :semitones
# scale = :mel_htk

usemfcc = false
# usemfcc = true
usef0 = false
# usef0 = true

sr = 8000
audioparams = (
    usemfcc = usemfcc,
    usef0 = usef0,
    sr = sr,
    nfft = 512,
    scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    nbands = scale == :semitones ? 14 : 26,
    ncoeffs = scale == :semitones ? 7 : 13,
    freq_range = (300, round(Int, sr / 2)),
    db_scale = usemfcc ? false : true,
)

memguard = false;
# memguard = true;
n_elems = 15;

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]

@assert experiment in avail_exp "Unknown type of experiment: $experiment."

findhealthy = y -> findall(x -> x == "Healthy", y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
findsick = y -> findall(x -> x == String(experiment), y)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"
memguard && begin filename *= string("_memguard") end

destpath = "results/propositional"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36);
r_select = r"\e\[\d+m(.*?)\e\[0m";

### Audio features extraction function
This function is called for every audio sample and extract 25 features:
14 bands of the mel spectrogram,
11 spectral features: centroid, crest, entropy, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [3]:
function afe(x::AbstractVector{Float64}, audioparams::NamedTuple; get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = audioparams.sr
    norm = true
    speech_detection = false
    # stft module
    nfft = audioparams.nfft
    win_type = (:hann, :periodic)
    win_length = audioparams.nfft
    overlap_length = round(Int, audioparams.nfft / 2)
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = audioparams.nbands
    scale = audioparams.scale               # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = audioparams.freq_range
    # mel spectrogram module
    db_scale = audioparams.db_scale

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        nfft=nfft,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.data.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    if audioparams.usemfcc
        # mfcc module
        ncoeffs = audioparams.ncoeffs
        rectification = :log                    # :log, :cubic_root
        dither = true

        mfcc = get_mfcc(
            source=melspec,
            ncoeffs=ncoeffs,
            rectification=rectification,
            dither=dither,
        );
    end

    if audioparams.usef0
        # f0 module
        method = :nfc
        f0_range = (50, 400)

        f0 = get_f0(
            source=stftspec,
            method=method,
            freq_range=f0_range
        );
    end

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    hcat(
        filter(!isnothing, [
            melspec.spec',
            audioparams.usemfcc ? mfcc.mfcc' : nothing,
            audioparams.usef0 ? f0.f0 : nothing,
            spect.centroid,
            spect.crest,
            spect.entropy,
            spect.flatness,
            spect.flux,
            spect.kurtosis,
            spect.rolloff,
            spect.skewness,
            spect.decrease,
            spect.slope,
            spect.spread
        ])...
    )    
end

afe (generic function with 1 method)

### Prepare dataset for analysis

In [4]:
d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

memguard && begin
    cat2 = round(Int, length(y)/2)
    indices = [1:n_elems; cat2:cat2+n_elems-1]
    x = x[indices, :]
    y = y[indices]
end

freq = round.(Int, afe(x[1, :audio], audioparams; get_only_melfreq=true))

catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.nbands],
        audioparams.usemfcc ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.ncoeffs] : String[],
        audioparams.usef0 ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)
    
catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

if !loadset
    @info("Build dataset...")

    X = DataFrame([name => Float64[] for name in [match(r_select, v)[1] for v in variable_names]])
    audiofeats = [afe(row[:audio], audioparams) for row in eachrow(x)]
    push!(X, vcat([vcat([map(func, eachcol(row)) for func in catch9]...) for row in audiofeats])...)

    yc = CategoricalArray(y);

    train_ratio = 0.8

    train, test = partition(eachindex(yc), train_ratio, shuffle=true)
    X_train, y_train = X[train, :], yc[train]
    X_test, y_test = X[test, :], yc[test]
    save(dsfile, Dict("X_test" => X_test, "y_test" => y_test))

    println("Training set size: ", size(X_train), " - ", length(y_train))
    println("Test set size: ", size(X_test), " - ", length(y_test))
end

┌ Info: Build dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:42


Training set size: (440, 225) - 440
Test set size: (110, 225) - 110


### Train a model

In [5]:
if !loadset
    learned_dt_tree = begin
        Tree = MLJ.@load DecisionTreeClassifier pkg=DecisionTree
        model = Tree(max_depth=-1, )
        mach = machine(model, X_train, y_train)
        fit!(mach)
        fitted_params(mach).tree
    end
end

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/paso/.julia/packages/MLJModels/8W54X/src/loading.jl:159


import MLJDecisionTreeInterface ✔


┌ Info: Training machine(DecisionTreeClassifier(max_depth = -1, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


min(mel5=711Hz) < -3.84
├─ max(cntrd) < 1065.0
│  ├─ mean(flatn) < 0.005786
│  │  ├─ COPD (39/39)
│  │  └─ min(mel14=3366Hz) < -5.871
│  │     ├─ 3ac(decrs) < 0.02957
│  │     │  ├─ max(rllff) < 2734.0
│  │     │  │  ⋮
│  │     │  │  
│  │     │  └─ max(mel9=1419Hz) < -4.537
│  │     │     ⋮
│  │     │     
│  │     └─ min(mel8=1194Hz) < -4.029
│  │        ├─ COPD (33/33)
│  │        └─ Healthy (2/2)
│  └─ min(mel13=2832Hz) < -8.151
│     ├─ COPD (5/5)
│     └─ min(mel1=357Hz) < -3.709
│        ├─ std(mel10=1687Hz) < 0.1122
│        │  ├─ mean(mel8=1194Hz) < -5.057
│        │  │  ⋮
│        │  │  
│        │  └─ min(entrp) < 0.6284
│        │     ⋮
│        │     
│        └─ min(mel8=1194Hz) < -4.268
│           ├─ min(crest) < 11.98
│           │  ⋮
│           │  
│           └─ Healthy (16/16)
└─ mean(mel12=2383Hz) < -4.165
   ├─ max(mel9=1419Hz) < -2.058
   │  ├─ std(mel1=357Hz) < 0.6095
   │  │  ├─ COPD (38/38)
   │  │  └─ min(entrp) < 0.4223
   │  │     ├─ COPD (1/1)
   │  │    

### Model inspection & rule study

In [6]:
if !loadset
    sole_dt = solemodel(learned_dt_tree)
    # Make test instances flow into the model, so that test metrics can, then, be computed.
    apply!(sole_dt, X_test, y_test);
    # Save solemodel to disk
    save(jld2file, Dict("sole_dt" => sole_dt))
else
    @info("Load dataset...")
    d = jldopen(dsfile)
    X_test = d["X_test"]
    y_test = d["y_test"]
    close(d)
    d = jldopen(jld2file)
    sole_dt = d["sole_dt"]
    close(d)
end
# Print Sole model
printmodel(sole_dt; show_metrics = true, variable_names_map = variable_names);

▣ min(mel5=711Hz) < -3.8398306837298675
├✔ max(cntrd) < 1064.6875266452453
│├✔ mean(flatn) < 0.005785640626093488
││├✔ COPD : (ninstances = 6, ncovered = 6, confidence = 1.0, lift = 1.0)
││└✘ min(mel14=3366Hz) < -5.871457993702041
││ ├✔ 3ac(decrs) < 0.029569046870084936
││ │├✔ max(rllff) < 2734.375
││ ││├✔ Healthy : (ninstances = 5, ncovered = 5, confidence = 0.6, lift = 1.0)
││ ││└✘ std(mel7=1005Hz) < 0.5570114664467685
││ ││ ├✔ Healthy : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
││ ││ └✘ COPD : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
││ │└✘ max(mel9=1419Hz) < -4.537417154234595
││ │ ├✔ Healthy : (ninstances = 2, ncovered = 2, confidence = 0.5, lift = 1.0)
││ │ └✘ COPD : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
││ └✘ min(mel8=1194Hz) < -4.0288742621675295
││  ├✔ COPD : (ninstances = 14, ncovered = 14, confidence = 0.93, lift = 1.0)
││  └✘ Healthy : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
│└✘ min(mel13=283

### Extract rules that are at least as good as a random baseline model

In [7]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map = variable_names);

▣ (min(mel5=711Hz) < -3.8398306837298675) ∧ (max(cntrd) < 1064.6875266452453) ∧ (mean(flatn) < 0.005785640626093488)  ↣  COPD : (ninstances = 110, ncovered = 6, coverage = 0.05, confidence = 1.0, lift = 2.04, natoms = 3)
▣ (min(mel5=711Hz) < -3.8398306837298675) ∧ (max(cntrd) < 1064.6875266452453) ∧ (¬(mean(flatn) < 0.005785640626093488)) ∧ (min(mel14=3366Hz) < -5.871457993702041) ∧ (3ac(decrs) < 0.029569046870084936) ∧ (max(rllff) < 2734.375)  ↣  Healthy : (ninstances = 110, ncovered = 5, coverage = 0.05, confidence = 0.6, lift = 1.18, natoms = 6)
▣ (min(mel5=711Hz) < -3.8398306837298675) ∧ (max(cntrd) < 1064.6875266452453) ∧ (¬(mean(flatn) < 0.005785640626093488)) ∧ (min(mel14=3366Hz) < -5.871457993702041) ∧ (3ac(decrs) < 0.029569046870084936) ∧ (¬(max(rllff) < 2734.375)) ∧ (std(mel7=1005Hz) < 0.5570114664467685)  ↣  Healthy : (ninstances = 110, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 1.96, natoms = 7)
▣ (min(mel5=711Hz) < -3.8398306837298675) ∧ (max(cntrd) < 1064.687

### Simplify rules while extracting and prettify result

In [8]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map = variable_names);

▣ (min(mel5=711Hz) < -3.84) ∧ (max(cntrd) < 1064.69) ∧ (mean(flatn) < 0.01)  ↣  COPD : (ninstances = 110, ncovered = 6, coverage = 0.05, confidence = 1.0, lift = 2.04, natoms = 3)
▣ (min(mel5=711Hz) < -3.84) ∧ (max(cntrd) < 1064.69) ∧ (mean(flatn) ≥ 0.01) ∧ (min(mel14=3366Hz) < -5.87) ∧ (3ac(decrs) < 0.03) ∧ (max(rllff) < 2734.38)  ↣  Healthy : (ninstances = 110, ncovered = 5, coverage = 0.05, confidence = 0.6, lift = 1.18, natoms = 6)
▣ (min(mel5=711Hz) < -3.84) ∧ (max(cntrd) < 1064.69) ∧ (mean(flatn) ≥ 0.01) ∧ (min(mel14=3366Hz) < -5.87) ∧ (3ac(decrs) < 0.03) ∧ (max(rllff) ≥ 2734.38) ∧ (std(mel7=1005Hz) < 0.56)  ↣  Healthy : (ninstances = 110, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 1.96, natoms = 7)
▣ (min(mel5=711Hz) < -3.84) ∧ (max(cntrd) < 1064.69) ∧ (mean(flatn) ≥ 0.01) ∧ (min(mel14=3366Hz) < -5.87) ∧ (3ac(decrs) ≥ 0.03) ∧ (max(mel9=1419Hz) ≥ -4.54)  ↣  COPD : (ninstances = 110, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 2.04, natoms = 6)
▣ (min(mel5

### Directly access rule metrics

In [9]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

15-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 110, ncovered = 6, coverage = 0.05454545454545454, confidence = 1.0, lift = 2.037037037037037, natoms = 3)
 (ninstances = 110, ncovered = 5, coverage = 0.045454545454545456, confidence = 0.6, lift = 1.1785714285714286, natoms = 6)
 (ninstances = 110, ncovered = 1, coverage = 0.00909090909090909, confidence = 1.0, lift = 1.9642857142857144, natoms = 7)
 (ninstances = 110, ncovered = 1, coverage = 0.00909090909090909, confidence = 1.0, lift = 2.037037037037037, natoms = 6)
 (ninstances = 110, ncovered = 14, coverage = 0.12727272727272726, confidence = 0.9285714285714286, lift = 1.8915343915343916, natoms = 5)
 (ninstances = 110, ncovered = 1, coverage = 0.00909090909090909, confidence = 1.0, lift = 1.9642857142857144, natoms = 5)
 (ninstances = 110, ncovered = 1, coverage = 0.00909090909090909, confidence = 1.0, lift = 2.037037037037037,

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [10]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map = variable_names);

▣ (min(mel5=711Hz) < -3.8398306837298675) ∧ (max(cntrd) < 1064.6875266452453) ∧ (mean(flatn) ≥ 0.005785640626093488) ∧ (min(mel14=3366Hz) ≥ -5.871457993702041) ∧ (min(mel8=1194Hz) ≥ -4.0288742621675295)  ↣  Healthy : (ninstances = 110, ncovered = 1, coverage = 0.00909090909090909, confidence = 1.0, lift = 1.9642857142857144, natoms = 5, height = 4)
▣ (min(mel5=711Hz) < -3.8398306837298675) ∧ (max(cntrd) < 1064.6875266452453) ∧ (mean(flatn) ≥ 0.005785640626093488) ∧ (min(mel14=3366Hz) < -5.871457993702041) ∧ (3ac(decrs) < 0.029569046870084936) ∧ (max(rllff) ≥ 2734.375) ∧ (std(mel7=1005Hz) < 0.5570114664467685)  ↣  Healthy : (ninstances = 110, ncovered = 1, coverage = 0.00909090909090909, confidence = 1.0, lift = 1.9642857142857144, natoms = 7, height = 6)
▣ (min(mel5=711Hz) < -3.8398306837298675) ∧ (max(cntrd) ≥ 1064.6875266452453) ∧ (min(mel13=2832Hz) ≥ -8.151272401821737) ∧ (min(mel1=357Hz) < -3.7087625772680517) ∧ (std(mel10=1687Hz) ≥ 0.11216417602580012) ∧ (min(entrp) ≥ 0.6283783550

### Pretty table of rules and their metrics

In [11]:
metricstable(interesting_rules; variable_names_map = variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────┬──────────┬────────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                                                                                              Antecedent │ Consequent │ ninstances │ ncovered │   coverage │ confidence │    lift │ natoms │ height │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

# Inspect features

In [12]:
interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map=variable_names);

▣ (min(mel5=711Hz) < -3.84) ∧ (max(cntrd) < 1064.69) ∧ (mean(flatn) ≥ 0.01) ∧ (min(mel14=3366Hz) ≥ -5.87) ∧ (min(mel8=1194Hz) < -4.03)  ↣  COPD : (ninstances = 110, ncovered = 14, coverage = 0.13, confidence = 0.93, lift = 1.89, natoms = 5)
▣ (min(mel5=711Hz) < -3.84) ∧ (max(cntrd) ≥ 1064.69) ∧ (min(mel13=2832Hz) ≥ -8.15) ∧ (min(mel1=357Hz) < -3.71) ∧ (std(mel10=1687Hz) ≥ 0.11) ∧ (min(entrp) < 0.63)  ↣  Healthy : (ninstances = 110, ncovered = 45, coverage = 0.41, confidence = 0.93, lift = 1.83, natoms = 6)
▣ (min(mel5=711Hz) ≥ -3.84) ∧ (mean(mel12=2383Hz) ≥ -4.17)  ↣  COPD : (ninstances = 110, ncovered = 19, coverage = 0.17, confidence = 1.0, lift = 2.04, natoms = 2)


In [13]:
interesting_features = unique(SoleData.feature.(SoleLogics.value.(vcat(SoleLogics.atoms.(i.antecedent for i in interesting_rules)...))))
interesting_variables = sort(SoleData.i_variable.(interesting_features))

10-element Vector{Symbol}:
 Symbol("max(cntrd)")
 Symbol("mean(flatn)")
 Symbol("mean(mel12=2383Hz)")
 Symbol("min(entrp)")
 Symbol("min(mel13=2832Hz)")
 Symbol("min(mel14=3366Hz)")
 Symbol("min(mel1=357Hz)")
 Symbol("min(mel5=711Hz)")
 Symbol("min(mel8=1194Hz)")
 Symbol("std(mel10=1687Hz)")